In [230]:
import pandas as pd

In [231]:
# Import all excel files
df_materials = pd.read_excel("Data/BD Materiais.xlsx")
df_sales_jan_group = pd.read_excel("Data/VENDAS COM JAN 2023.xlsx")
df_sales_fev_group = pd.read_excel("Data/VENDAS COM FEV 2023.xlsx")
df_sales_jan_out = pd.read_excel("Data/VENDAS GRUPO JAN 2023.xlsx")
df_sales_fev_out = pd.read_excel("Data/VENDAS GRUPO FEV 2023.xlsx")

In [232]:
# Merge all orders into one df
df = pd.concat([df_sales_jan_group, df_sales_fev_group, df_sales_jan_out, df_sales_fev_out])

Let's take a look at the dataset columns and first rows

In [233]:
df.head()

,Cliente,Local Entrega,Data Factura,Material,Material (Desc),Unidade,Qtd Encomendada,Grupo?,Remessa
0,1128254,6346669,2023-01-06,126898,AGUA DAS PEDRAS SALGADAS 6x0.33 PET,UN,600.0,Não,NaN
1,1128254,6346669,2023-01-06,558440,COCA COLA LATA 0.33,UN,1080.0,Não,NaN
2,1128254,6346669,2023-01-06,202791,"COMPAL CLASSICO PESSEGO 0,20 GFA (15)",UN,900.0,Não,NaN
3,1128254,6346669,2023-01-06,202790,"COMPAL CLASSICO PERA 0,20 GFA (15)",UN,300.0,Não,NaN
4,1047551,6147807,2023-01-06,124867,ADOCANTE PO STICKS CX.250 RIO BRAVO,KI,1.0,Não,NaN


Translating columns names to english and removing one which will not be neccessary to the analysis

In [234]:
df = df.rename(columns = {
    "Cliente":"client",
    "Local Entrega":"delivery_place",
    "Data Factura":"date",
    "Material":"product",
    "Material (Desc)":"product_description",
    "Unidade":"measure",
    "Qtd Encomendada":"quantity",
    "Grupo?":"is_internal_client"
    }).drop(columns=["Remessa"]).reset_index(drop=True)

After checking, we noticed some rows don't have a defined quantity. We will drop those, since we already have 500k+ rows.

In [235]:
df.sort_values("quantity").tail()

,client,delivery_place,date,product,product_description,measure,quantity,is_internal_client
648177,1002554,6003780,2023-02-28,557160,ENCORES,NaN,NaN,NaN
648178,1002554,6003780,2023-02-28,123000,TARA CAIXA PLAST NOVA F/L,NaN,NaN,NaN
648179,1002554,6003780,2023-02-28,551112,CEBOLA,NaN,NaN,NaN
648180,1002554,6003780,2023-02-28,551117,PIMENTOS VERDES,NaN,NaN,NaN
648181,1002554,6003780,2023-02-28,123000,TARA CAIXA PLAST NOVA F/L,NaN,NaN,NaN


In [236]:
df = df.dropna().reset_index(drop=True)

After checking the product best seller, we verify it is a box used to transport other products, that is only registered for control purposes. Let's drop those lines.

In [237]:
df['product'].value_counts()

123000    31327
551116     5653
551107     5070
551115     4655
127978     4426
          ...  
632053        1
576728        1
569453        1
637140        1
571074        1
Name: product, Length: 4687, dtype: int64

In [238]:
df[df["product"] == 123000]["product_description"].unique()

array(['TARA CAIXA PLAST NOVA F/L'], dtype=object)

In [239]:
df = df[df["client"] != 123000].reset_index(drop=True)

Now we want to standardize client codes that start with letter 'E'. These correspond to workers at the company that are able to buy products from the warehouse. As the delivery place is the warehouse itself, workers pick their orders there, we attribute one unique code (-1) to all workers.

In [240]:
df.loc[df['client'].str.startswith('E', na=False), ['client', 'delivery_place']] = -1

After these modifications, we are changing now the types of some columns.

In [241]:
df['client'] = df['client'].astype(int)
df['delivery_place'] = df['delivery_place'].astype(int)

Now let's take a look at the excel file with information about each product

In [242]:
df_materials.head()

,Código,Area\nPreparação,Famíla,Sub Famíla
0,126898,AMB,Bebidas,Aguas Minerais
1,558440,AMB,Bebidas,Refrigerantes
2,202791,AMB,Bebidas,Sumos
3,202790,AMB,Bebidas,Sumos
4,124867,AMB,Mercearia,Acucar - Adocantes


Merge this data with the main dataset and rename the new columns

In [243]:
df = df.merge(df_materials, left_on='product', right_on='Código').rename(columns = {
    "Area\nPreparação":"warehouse_zone",
    "Famíla":"product_type",
    "Sub Famíla":"product_subtype"
    }).drop(columns=["Código"])

Assign an unique ID to all rows with the same delivery_place and date

In [244]:
df['delivery_place_date'] = df['delivery_place'].astype(str) + '_' + df['date'].astype(str)
df['order_id'] = pd.factorize(df['delivery_place_date'])[0] + 1
df = df.drop('delivery_place_date', axis=1)

Here is the final version of the dataset

In [245]:
df.head()

,client,delivery_place,date,product,product_description,measure,quantity,is_internal_client,warehouse_zone,product_type,product_subtype,order_id
0,1128254,6346669,2023-01-06,126898,AGUA DAS PEDRAS SALGADAS 6x0.33 PET,UN,600.0,Não,AMB,Bebidas,Aguas Minerais,1
1,1001096,6001131,2023-01-03,126898,AGUA DAS PEDRAS SALGADAS 6x0.33 PET,UN,120.0,Não,AMB,Bebidas,Aguas Minerais,2
2,1001096,6001131,2023-01-17,126898,AGUA DAS PEDRAS SALGADAS 6x0.33 PET,UN,120.0,Não,AMB,Bebidas,Aguas Minerais,3
3,1121833,6358142,2023-01-04,126898,AGUA DAS PEDRAS SALGADAS 6x0.33 PET,UN,12.0,Não,AMB,Bebidas,Aguas Minerais,4
4,1122758,6328488,2023-01-04,126898,AGUA DAS PEDRAS SALGADAS 6x0.33 PET,UN,60.0,Não,AMB,Bebidas,Aguas Minerais,5


### Metadata

Dataset description:
- client: internal code of the client
- delivery_place: internal code for a client delivery place (each client can have many delivery places)
- date: date of the order
- product: internal code of the product
- product_description
- measure: unit used to measure quantity number
- quantity
- is_internal_client: if the client belongs to the company group, i.e. has the same holding company
- warehouse_zone: internal code of the zone the product is located in the warehouse
- product_type
- product_subtype

In [246]:
# df.to_csv("orders.csv")

In [215]:
test = df[df["order_id"] == 8] # df[(df["order_id"] >= 1) & (df["order_id"] <= 8)]
test

,client,delivery_place,date,product,product_description,measure,quantity,is_internal_client,warehouse_zone,product_type,product_subtype,order_id
7,1007396,6323876,2023-01-03,126898,AGUA DAS PEDRAS SALGADAS 6x0.33 PET,UN,48.00,Não,AMB,Bebidas,Aguas Minerais,8
9960,1007396,6323876,2023-01-03,123000,TARA CAIXA PLAST NOVA F/L,UN,0.00,Não,AMB,Taras,Taras (Outras),8
9961,1007396,6323876,2023-01-03,123000,TARA CAIXA PLAST NOVA F/L,UN,0.00,Não,AMB,Taras,Taras (Outras),8
9962,1007396,6323876,2023-01-03,123000,TARA CAIXA PLAST NOVA F/L,UN,0.00,Não,AMB,Taras,Taras (Outras),8
9963,1007396,6323876,2023-01-03,123000,TARA CAIXA PLAST NOVA F/L,UN,0.00,Não,AMB,Taras,Taras (Outras),8
...,...,...,...,...,...,...,...,...,...,...,...,...
394648,1007396,6323876,2023-01-03,571758,TIRA GORDURAS MISTOLIN 545 ML,UN,10.00,Não,NAL,Nao Alimentares,Detergentes Liquidos - Amaciadores,8
394665,1007396,6323876,2023-01-03,572407,QUEIJO QUEIJAR POUSADAS AMANT. +/- 600G,KG,1.25,Não,REF,Mercearia,Queijos,8
394674,1007396,6323876,2023-01-03,587787,GELADO CARTE D'OR MERENG. FRT VERM 900ML,UN,4.00,Não,CON,Mercearia,Gelados,8
394684,1007396,6323876,2023-01-03,554421,PEIXE ESPADA BRANCO FRESCO,KG,6.00,Não,PEX,Peixe,Peixe Fresco,8


In [211]:
dots

,zone,x,y
0,AMB,4.0,4.30
1,REF,1.0,3.75
2,CON,1.0,1.50
3,NAL,3.5,3.00
4,DET,4.8,3.00
5,BAT,6.2,4.30
6,FRL,3.0,1.00
7,SAL,6.3,3.00
8,CAR,8.9,2.80
9,MAT,8.9,4.30


In [228]:
import numpy as np
import pandas as pd
import time

start_time = time.time()

dots = pd.DataFrame({"zone":['AMB', 'REF', 'CON', 'NAL', 'DET', 'BAT', 'FRL', 'SAL', 'CAR', 'MAT', 'FRE', 'PEX'],
                     "x":[4, 1, 1, 3.5, 4.8, 6.2, 3, 6.3, 8.9, 8.9, 5.7, 8.9],
                     "y":[4.3, 3.75, 1.5, 3, 3, 4.3, 1, 3, 2.8, 4.3, 1, 1]})
dots.loc[len(dots)] = ["START", 0, 0]

# Function for calculating distance between zones
def calculate_distance(zone1, zone2):
    x1, y1 = float(dots[dots["zone"] == zone1]["x"]), float(dots[dots["zone"] == zone1]["y"]) # dots[zone1][0], dots[zone1][1]
    x2, y2 = float(dots[dots["zone"] == zone2]["x"]), float(dots[dots["zone"] == zone2]["y"])
    distance = math.sqrt((x2 - x1)**2 + (y2 - y1)**2)
    return distance

def get_shortest_distance_zone(zone_list):
    distances = []
    for zone in zone_list:
        distances.append(calculate_distance("START", zone))
    return zone_list[np.argmin(distances)]

# Parameters for Q-learning
alpha = 0.1  # Learning rate
gamma = 0.99  # Discount factor
epsilon = 0.1  # Exploration rate
n_episodes = 10 #1000  # Number of episodes

# Get the unique order ids and zones
order_ids = test['order_id'].unique()
zones = test['warehouse_zone'].unique()
min_order_id = np.min(order_ids)

# Initialize the Q-table
num_zones = len(order_ids)
q_table = np.zeros((len(order_ids), len(zones), len(zones)))

for episode in range(n_episodes):
    for order_id in order_ids:
        order = test[test['order_id'] == order_id].sort_values(by='date')
        zone_list = order['warehouse_zone'].tolist()

        for i in range(len(zone_list) - 1):
            current_zone = zone_list[i]
            next_zone = zone_list[i + 1]

            current_zone_idx = np.where(zones == current_zone)[0][0]
            next_zone_idx = np.where(zones == next_zone)[0][0]

            # Choose action (next zone) using epsilon-greedy strategy
            if np.random.random() < epsilon:
                action_idx = np.random.randint(num_zones)
            else:
                action_idx = np.argmax(q_table[order_id - min_order_id, current_zone_idx, :])

            # Calculate the reward based on the distance to the chosen zone
            reward = -calculate_distance(current_zone, zones[min(action_idx, len(zones)-1)])

            # Update Q-table
            q_table[order_id - min_order_id, current_zone_idx, min(action_idx, len(zones)-1)] = q_table[order_id - min_order_id, current_zone_idx, min(action_idx, len(zones)-1)] + alpha * (reward + gamma * np.max(q_table[order_id - min_order_id, min(action_idx, len(zones)-1), :]) - q_table[order_id - min_order_id, current_zone_idx, min(action_idx, len(zones)-1)])

# Print the optimal picking order
for order_id in order_ids:
    order = test[test['order_id'] == order_id].sort_values(by='date')
    zone_list = order['warehouse_zone'].tolist()

    first_zone = get_shortest_distance_zone(zone_list)
    optimal_order = [first_zone]

    for i in range(len(zone_list) - 1):
        current_zone = zone_list[i + 1]
        current_zone_idx = np.where(zones == current_zone)[0][0]

        # Choose the best action based on the Q-table
        action_idx = np.argmax(q_table[order_id - min_order_id, current_zone_idx, :])

        if zones[action_idx] not in optimal_order:
            optimal_order.append(zones[action_idx])

    print(f"Optimal picking order for order {order_id}: {optimal_order}")

end_time = time.time()

print("Running time:", end_time - start_time, "seconds")

Optimal picking order for order 8: ['CON', 'NAL', 'AMB', 'REF', 'FRL', 'DET', 'CAR', 'PEX']
Running time: 2.4491798877716064 seconds


The ε-greedy strategy is a trade-off between exploration (trying out new actions to find better solutions) and exploitation (using the best-known action based on the current Q-table). The parameter epsilon (ε) determines the probability of choosing a random action instead of the best-known action.

Here's a breakdown of the code snippet:

 - if np.random.random() < epsilon:: Generate a random number between 0 and 1. If the random number is less than epsilon (ε), the agent will explore by choosing a random action (zone) to visit next.
 - action_idx = np.random.randint(num_zones): If the agent is in exploration mode (random number is less than epsilon), it chooses a random action (zone) to visit next. np.random.randint(num_zones) generates a random integer between 0 and num_zones-1 representing the index of the next zone in the zones array.
 - else:: If the agent is in exploitation mode (random number is greater than or equal to epsilon), it will choose the best-known action based on the current Q-table.
  - action_idx = np.argmax(q_table[order_id - min_order_id, current_zone_idx, :]): In exploitation mode, the agent selects the action (zone) with the highest Q-value for the current state (order_id and zone). np.argmax() finds the index of the maximum Q-value for the current state in the Q-table.

By using the ε-greedy strategy, the agent can balance exploration and exploitation to find the optimal picking order over time.

- alpha (learning rate): The learning rate determines how much the agent updates its Q-table after observing a new reward. A higher learning rate means that the agent will put more weight on the new information it receives, whereas a lower learning rate means that the agent will update its Q-table more conservatively. The learning rate should be between 0 and 1, with typical values ranging from 0.1 to 0.9.

 - gamma (discount factor): The discount factor is used to balance the importance of immediate rewards versus future rewards. A higher discount factor means that the agent will prioritize long-term rewards more, while a lower discount factor will make the agent more focused on immediate rewards. The discount factor should also be between 0 and 1, and typical values range from 0.9 to 0.99.
 
The Q-table update is based on the Q-learning update rule:

##### Q(s, a) = Q(s, a) + α * (r + γ * max(Q(s', a')) - Q(s, a))


where:

Q(s, a): Current Q-value for the state *s* and action *a*

α (alpha): Learning rate

r: Reward received after taking action *a*

γ (gamma): Discount factor

max(Q(s', a')): Maximum Q-value for the next state *s'* and all possible actions a'

The update rule adjusts the Q-value for the current state s and action a based on the observed reward r and the maximum Q-value for the next state s'. The learning rate alpha controls how much the agent updates its Q-value, and the discount factor gamma determines the importance of future rewards in the update.

# OLD CODE

In [207]:
# Code 1st try primeira zona definida
for order_id in order_ids:
    order = test[test['order_id'] == order_id].sort_values(by='date')
    zone_list = order['warehouse_zone'].tolist()

    first_zone = get_shortest_distance_zone(zone_list)
    optimal_order = [first_zone]
    zone_list.remove(first_zone)

    for i in range(len(zone_list)):
        current_zone = optimal_order[-1]
        current_zone_idx = np.where(zones == current_zone)[0][0]
        
# Original  code
for order_id in order_ids:
    order = test[test['order_id'] == order_id].sort_values(by='date')
    zone_list = order['warehouse_zone'].tolist()

    optimal_order = [zone_list[0]]
    for i in range(len(zone_list) - 1):
        current_zone = zone_list[i]
        current_zone_idx = np.where(zones == current_zone)[0][0]

In [94]:
import numpy as np
import pandas as pd
import time

start_time = time.time()

# Placeholder function for calculating distance between zones
def calculate_distance(zone1, zone2):
    # Replace this function with your actual distance calculation based on your warehouse map
    return np.random.randint(1, 100)

# Parameters for Q-learning
alpha = 0.1  # Learning rate
gamma = 0.99  # Discount factor
epsilon = 0.1  # Exploration rate
n_episodes = 1000  # Number of episodes

# Get the unique order ids and zones
order_ids = test['order_id'].unique()
zones = test['warehouse_zone'].unique()

# Initialize the Q-table
q_table = np.zeros((len(order_ids), len(zones), len(zones)))

for episode in range(n_episodes):
    for order_id in order_ids:
        order = test[test['order_id'] == order_id].sort_values(by='date')
        zone_list = order['warehouse_zone'].tolist()

        current_zone = zone_list[0]  # Start at the first zone
        for next_zone in zone_list[1:]:
            current_zone_idx = np.where(zones == current_zone)[0][0]
            next_zone_idx = np.where(zones == next_zone)[0][0]

            # Choose action (next zone) using epsilon-greedy strategy
            if np.random.random() < epsilon:
                action = np.random.choice(zones)
            else:
                action = zones[np.argmax(q_table[order_id - 1, current_zone_idx, :])]

            action_idx = np.where(zones == action)[0][0]

            # Calculate the reward based on the distance to the chosen zone
            reward = -calculate_distance(current_zone, action)

            # Update Q-table
            q_table[order_id - 1, current_zone_idx, action_idx] = q_table[order_id - 1, current_zone_idx, action_idx] + alpha * (reward + gamma * np.max(q_table[order_id - 1, action_idx, :]) - q_table[order_id - 1, current_zone_idx, action_idx])

            # Move to the next zone
            current_zone = action

# Print the optimal picking order
for order_id in order_ids:
    order = test[test['order_id'] == order_id].sort_values(by='date')
    zone_list = order['warehouse_zone'].tolist()
    optimal_order = []

    current_zone = zone_list[0]  # Start at the first zone
    optimal_order.append(current_zone)
    for next_zone in zone_list[1:]:
        current_zone_idx = np.where(zones == current_zone)[0][0]

        # Choose the best action based on the Q-table
        action = zones[np.argmax(q_table[order_id - 1, current_zone_idx, :])]

        optimal_order.append(action)
        current_zone = action

    print(f"Optimal picking order for order {order_id}: {optimal_order}")

end_time = time.time()

print("Running time:", end_time - start_time, "seconds")

Optimal picking order for order 1: ['AMB', 'REF', 'REF', 'REF']
Optimal picking order for order 2: ['AMB', 'CON', 'REF', 'CON', 'REF', 'CON', 'REF', 'CON', 'REF', 'CON', 'REF', 'CON', 'REF', 'CON', 'REF', 'CON', 'REF', 'CON']
Optimal picking order for order 3: ['AMB', 'CON', 'CON', 'CON', 'CON', 'CON', 'CON', 'CON', 'CON', 'CON', 'CON', 'CON', 'CON', 'CON', 'CON']
Optimal picking order for order 4: ['AMB', 'CON', 'REF', 'REF', 'REF', 'REF']
Optimal picking order for order 5: ['AMB', 'CON', 'REF', 'AMB', 'CON', 'REF', 'AMB', 'CON', 'REF', 'AMB']
Running time: 7.625851154327393 seconds


In [71]:
q_table

array([[[-1278.73290934, -1278.25895806, -1278.93203558],
        [-1272.96877375, -1271.6051068 , -1267.6669187 ],
        [-1269.39525125, -1268.92888487, -1266.09624144]],

       [[-4229.62055288, -4218.59088129, -4224.61760135],
        [-4224.23652117, -4230.55562116, -4228.49722491],
        [-4223.97943491, -4223.18278581, -4225.3539628 ]],

       [[-3918.35251361, -3916.12933503, -3918.38587773],
        [-3926.3928069 , -3924.79093659, -3924.31316436],
        [-3918.75785925, -3918.57634404, -3920.27539476]],

       [[-2036.04623971, -2032.93480625, -2037.71584419],
        [-2024.00940171, -2016.40413431, -2012.92140907],
        [-2022.32522367, -2019.23495009, -2023.17411531]],

       [[-3134.49466384, -3137.16579653, -3133.01907416],
        [-3137.24752953, -3136.48144784, -3135.07219341],
        [-3133.82657894, -3135.91687235, -3137.18716459]]])

In [132]:
import numpy as np
import pandas as pd
import time

start_time = time.time()

dots = pd.DataFrame({"zone":['AMB', 'REF', 'CON', 'NAL', 'DET', 'BAT', 'FRL', 'SAL', 'CAR', 'MAT', 'FRE', 'PEX'],
                     "x":[4, 1, 1, 3.5, 4.8, 6.2, 3, 6.3, 8.9, 8.9, 5.7, 8.9],
                     "y":[4.3, 3.75, 1.5, 3, 3, 4.3, 1, 3, 2.8, 4.3, 1, 1]})

# Function for calculating distance between zones
def calculate_distance(zone1, zone2):
    x1, y1 = float(dots[dots["zone"] == zone1]["x"]), float(dots[dots["zone"] == zone1]["y"]) # dots[zone1][0], dots[zone1][1]
    x2, y2 = float(dots[dots["zone"] == zone2]["x"]), float(dots[dots["zone"] == zone2]["y"])
    distance = math.sqrt((x2 - x1)**2 + (y2 - y1)**2)
    return distance

# Parameters for Q-learning
alpha = 0.1  # Learning rate
gamma = 0.99  # Discount factor
epsilon = 0.1  # Exploration rate
n_episodes = 1000  # Number of episodes

# Get the unique order ids and zones
order_ids = test['order_id'].unique()
zones = test['warehouse_zone'].unique()

# Initialize the Q-table
num_zones = len(order_ids)
q_table = np.zeros((len(order_ids), len(zones), len(zones)))

for episode in range(n_episodes):
    for order_id in order_ids:
        order = test[test['order_id'] == order_id].sort_values(by='date')
        zone_list = order['warehouse_zone'].tolist()

        for i in range(len(zone_list) - 1):
            current_zone = zone_list[i]
            next_zone = zone_list[i + 1]

            current_zone_idx = np.where(zones == current_zone)[0][0]
            next_zone_idx = np.where(zones == next_zone)[0][0]

            # Choose action (next zone) using epsilon-greedy strategy
            if np.random.random() < epsilon:
                action_idx = np.random.randint(num_zones)
            else:
                action_idx = np.argmax(q_table[order_id - 1, current_zone_idx, :])

            # Calculate the reward based on the distance to the chosen zone
            reward = -calculate_distance(current_zone, zones[min(action_idx, len(zones)-1)])

            # Update Q-table
            q_table[order_id - 1, current_zone_idx, min(action_idx, len(zones)-1)] = q_table[min(order_id - 1, len(zones)-1), current_zone_idx, min(action_idx, len(zones)-1)] + alpha * (reward + gamma * np.max(q_table[min(order_id - 1, len(zones)-1), min(action_idx, len(zones)-1), :]) - q_table[min(order_id - 1, len(zones)-1), current_zone_idx, min(action_idx, len(zones)-1)])

# Print the optimal picking order
for order_id in order_ids:
    order = test[test['order_id'] == order_id].sort_values(by='date')
    zone_list = order['warehouse_zone'].tolist()

    optimal_order = [zone_list[0]]
    for i in range(len(zone_list) - 1):
        current_zone = zone_list[i]
        current_zone_idx = np.where(zones == current_zone)[0][0]

        # Choose the best action based on the Q-table
        action_idx = np.argmax(q_table[order_id - 1, current_zone_idx, :])

        if zones[action_idx] not in optimal_order:
            optimal_order.append(zones[action_idx])

    print(f"Optimal picking order for order {order_id}: {optimal_order}")


end_time = time.time()

print("Running time:", end_time - start_time, "seconds")

Optimal picking order for order 1: ['AMB']
Optimal picking order for order 2: ['AMB', 'CON', 'REF']
Optimal picking order for order 3: ['AMB', 'CON', 'REF']
Optimal picking order for order 4: ['AMB', 'REF']
Optimal picking order for order 5: ['AMB', 'REF']
Optimal picking order for order 6: ['AMB', 'REF']
Optimal picking order for order 7: ['AMB']
Optimal picking order for order 8: ['AMB', 'NAL', 'CON', 'REF', 'FRL', 'DET', 'CAR', 'PEX']
Running time: 300.7600598335266 seconds


In [130]:
q_table

array([[[0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.]]])

In [108]:
min(action_idx, len(zones)-1)

2

In [116]:
import numpy as np
import pandas as pd
import math
import time

start_time = time.time()

dots = pd.DataFrame({"zone": ['AMB', 'REF', 'CON', 'NAL', 'DET', 'BAT', 'FRL', 'SAL', 'CAR', 'MAT', 'FRE', 'PEX'],
                     "x": [4, 1, 1, 3.5, 4.8, 6.2, 3, 6.3, 8.9, 8.9, 5.7, 8.9],
                     "y": [4.3, 3.75, 1.5, 3, 3, 4.3, 1, 3, 2.8, 4.3, 1, 1]})


# Function for calculating distance between zones
def calculate_distance(zone1, zone2):
    x1, y1 = float(dots[dots["zone"] == zone1]["x"]), float(dots[dots["zone"] == zone1]["y"])
    x2, y2 = float(dots[dots["zone"] == zone2]["x"]), float(dots[dots["zone"] == zone2]["y"])
    distance = math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)
    return distance


def greedy_picking_order(zone_list):
    remaining_zones = zone_list.copy()
    current_zone = remaining_zones.pop(0)
    optimal_order = [current_zone]

    while remaining_zones:
        min_distance = float('inf')
        next_zone = None

        for zone in remaining_zones:
            distance = calculate_distance(current_zone, zone)

            if distance < min_distance:
                min_distance = distance
                next_zone = zone

        optimal_order.append(next_zone)
        remaining_zones.remove(next_zone)
        current_zone = next_zone

    return optimal_order


order_ids = test['order_id'].unique()

# Print the optimal picking order
for order_id in order_ids:
    order = test[test['order_id'] == order_id].sort_values(by='date')
    zone_list = order['warehouse_zone'].tolist()

    optimal_order = greedy_picking_order(zone_list)
    print(f"Optimal picking order for order {order_id}: {optimal_order}")

end_time = time.time()

print("Running time:", end_time - start_time, "seconds")


Optimal picking order for order 8: ['AMB', 'AMB', 'AMB', 'AMB', 'AMB', 'AMB', 'AMB', 'AMB', 'AMB', 'AMB', 'AMB', 'AMB', 'AMB', 'AMB', 'AMB', 'AMB', 'AMB', 'AMB', 'AMB', 'AMB', 'AMB', 'AMB', 'AMB', 'AMB', 'AMB', 'AMB', 'AMB', 'AMB', 'AMB', 'AMB', 'AMB', 'AMB', 'AMB', 'AMB', 'AMB', 'AMB', 'AMB', 'AMB', 'AMB', 'AMB', 'AMB', 'AMB', 'NAL', 'NAL', 'NAL', 'NAL', 'NAL', 'NAL', 'NAL', 'NAL', 'NAL', 'NAL', 'NAL', 'NAL', 'NAL', 'DET', 'FRL', 'FRL', 'FRL', 'FRL', 'FRL', 'FRL', 'FRL', 'FRL', 'FRL', 'FRL', 'FRL', 'FRL', 'FRL', 'FRL', 'FRL', 'FRL', 'FRL', 'FRL', 'FRL', 'FRL', 'FRL', 'FRL', 'FRL', 'FRL', 'FRL', 'FRL', 'CON', 'CON', 'CON', 'CON', 'CON', 'CON', 'CON', 'CON', 'CON', 'CON', 'CON', 'CON', 'CON', 'CON', 'CON', 'CON', 'CON', 'CON', 'CON', 'CON', 'CON', 'CON', 'CON', 'CON', 'REF', 'REF', 'REF', 'REF', 'REF', 'REF', 'REF', 'REF', 'REF', 'REF', 'REF', 'REF', 'CAR', 'PEX', 'PEX']
Running time: 14.432393789291382 seconds


In [114]:
df[df['order_id'] == 8]

,client,delivery_place,date,product,product_description,measure,quantity,is_internal_client,warehouse_zone,product_type,product_subtype,order_id
7,1007396,6323876,2023-01-03,126898,AGUA DAS PEDRAS SALGADAS 6x0.33 PET,UN,48.00,Não,AMB,Bebidas,Aguas Minerais,8
9960,1007396,6323876,2023-01-03,123000,TARA CAIXA PLAST NOVA F/L,UN,0.00,Não,AMB,Taras,Taras (Outras),8
9961,1007396,6323876,2023-01-03,123000,TARA CAIXA PLAST NOVA F/L,UN,0.00,Não,AMB,Taras,Taras (Outras),8
9962,1007396,6323876,2023-01-03,123000,TARA CAIXA PLAST NOVA F/L,UN,0.00,Não,AMB,Taras,Taras (Outras),8
9963,1007396,6323876,2023-01-03,123000,TARA CAIXA PLAST NOVA F/L,UN,0.00,Não,AMB,Taras,Taras (Outras),8
...,...,...,...,...,...,...,...,...,...,...,...,...
394648,1007396,6323876,2023-01-03,571758,TIRA GORDURAS MISTOLIN 545 ML,UN,10.00,Não,NAL,Nao Alimentares,Detergentes Liquidos - Amaciadores,8
394665,1007396,6323876,2023-01-03,572407,QUEIJO QUEIJAR POUSADAS AMANT. +/- 600G,KG,1.25,Não,REF,Mercearia,Queijos,8
394674,1007396,6323876,2023-01-03,587787,GELADO CARTE D'OR MERENG. FRT VERM 900ML,UN,4.00,Não,CON,Mercearia,Gelados,8
394684,1007396,6323876,2023-01-03,554421,PEIXE ESPADA BRANCO FRESCO,KG,6.00,Não,PEX,Peixe,Peixe Fresco,8


In [126]:
import numpy as np
import pandas as pd
import math
import time
import tensorflow as tf

start_time = time.time()

dots = pd.DataFrame({"zone": ['AMB', 'REF', 'CON', 'NAL', 'DET', 'BAT', 'FRL', 'SAL', 'CAR', 'MAT', 'FRE', 'PEX'],
                     "x": [4, 1, 1, 3.5, 4.8, 6.2, 3, 6.3, 8.9, 8.9, 5.7, 8.9],
                     "y": [4.3, 3.75, 1.5, 3, 3, 4.3, 1, 3, 2.8, 4.3, 1, 1]})


# Function for calculating distance between zones
def calculate_distance(zone1, zone2):
    x1, y1 = float(dots[dots["zone"] == zone1]["x"]), float(dots[dots["zone"] == zone1]["y"])
    x2, y2 = float(dots[dots["zone"] == zone2]["x"]), float(dots[dots["zone"] == zone2]["y"])
    distance = math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)
    return distance


def build_q_network(input_shape, num_zones):
    model = tf.keras.models.Sequential([
        tf.keras.layers.Input(shape=input_shape),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(num_zones, activation=None)  # No activation in the output layer
    ])
    model.compile(optimizer=tf.keras.optimizers.Adam(), loss=tf.keras.losses.MeanSquaredError())
    return model



def encode_state(current_zone, remaining_zones, all_zones):
    current_zone_encoded = np.zeros(len(all_zones))
    current_zone_idx = np.where(all_zones == current_zone)[0][0]
    current_zone_encoded[current_zone_idx] = 1

    remaining_zones_encoded = np.zeros(len(all_zones))
    for zone in remaining_zones:
        zone_idx = np.where(all_zones == zone)[0][0]
        remaining_zones_encoded[zone_idx] = 1

    state = np.concatenate((current_zone_encoded, remaining_zones_encoded))
    state = state.reshape(1, -1)  # Reshape to match the expected input shape of the network
    return state



def train_q_network(q_network, order_id, current_zone, remaining_zones, next_zone, reward, gamma):
    state = encode_state(current_zone, remaining_zones, zones)
    next_state = encode_state(next_zone, [zone for zone in remaining_zones if zone != next_zone], zones)

    q_values = q_network.predict(state)
    next_q_values = q_network.predict(next_state)

    target_q_values = q_values.copy()
    target_q_values[0, np.where(zones == next_zone)[0][0]] = reward + gamma * np.max(next_q_values)

    q_network.fit(state, target_q_values, epochs=1, verbose=0)


order_ids = test['order_id'].unique()
zones = test['warehouse_zone'].unique()

input_shape = (2 * num_zones,)
num_zones = len(zones)
q_network = build_q_network(input_shape, num_zones)


alpha = 0.1
gamma = 0.99
epsilon = 0.1
n_episodes = 1000

for episode in range(n_episodes):
    for order_id in order_ids:
        order = test[test['order_id'] == order_id].sort_values(by='date')
        zone_list = order['warehouse_zone'].tolist()

        for i in range(len(zone_list) - 1):
            current_zone = zone_list[i]
            next_zone = zone_list[i + 1]

            remaining_zones = [zone for zone in zone_list if zone != current_zone]

            if np.random.random() < epsilon:
                next_zone_idx = np.random.randint(num_zones)
                next_zone = zones[next_zone_idx]
            else:
                state = encode_state(current_zone, remaining_zones, zones)
                q_values = q_network.predict(state)
                next_zone_idx = np.argmax(q_values)
                next_zone = zones[next_zone_idx]

            reward = -calculate_distance(current_zone, next_zone)

            train_q_network(q_network, order_id, current_zone, remaining_zones, next_zone, reward, gamma)

# Print the optimal picking order
for order_id in order_ids:
    order = test[test['order_id'] == order_id].sort_values(by='date')
    zone_list = order['warehouse_zone'].tolist()

    optimal_order = [zone_list[0]]
    current_zone = zone_list[0]
    remaining_zones = [zone for zone in zone_list if zone != current_zone]

    while remaining_zones:
        state = encode_state(current_zone, remaining_zones, zones)
        q_values = q_network.predict(state)
        next_zone_idx = np.argmax(q_values)
        next_zone = zones[next_zone_idx]

        optimal_order.append(next_zone)
        current_zone = next_zone
        remaining_zones = [zone for zone in remaining_zones if zone != current_zone]

    print(f"Optimal picking order for order {order_id}: {optimal_order}")

end_time = time.time()

print("Running time:", end_time - start_time, "seconds")


1/1 [==============================] - 0s 40ms/step


KeyboardInterrupt: 

In [124]:
build_q_network(input_shape, num_zones).predict(state)

ValueError: in user code:

    File "/Users/rodrigosimoes/opt/anaconda3/lib/python3.8/site-packages/keras/engine/training.py", line 2169, in predict_function  *
        return step_function(self, iterator)
    File "/Users/rodrigosimoes/opt/anaconda3/lib/python3.8/site-packages/keras/engine/training.py", line 2155, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Users/rodrigosimoes/opt/anaconda3/lib/python3.8/site-packages/keras/engine/training.py", line 2143, in run_step  **
        outputs = model.predict_step(data)
    File "/Users/rodrigosimoes/opt/anaconda3/lib/python3.8/site-packages/keras/engine/training.py", line 2111, in predict_step
        return self(x, training=False)
    File "/Users/rodrigosimoes/opt/anaconda3/lib/python3.8/site-packages/keras/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/Users/rodrigosimoes/opt/anaconda3/lib/python3.8/site-pack